# Simple example using Celery field, creates half-hourly footprint image files centered on tower location

In [6]:
import sys
sys.path.append("../../scripts/") 

import footprint_FFP_climatology as ffp
import pandas as pd

# Read in data and create datetimes for saving footprints

In [7]:
df = pd.read_csv('stable.csv')
# m = dict(yyyy='year', mm='month', day='day', HH_UTC='hour', MM='minute')
# v = df.iloc[:, :5].rename(columns=m)
# v['second'] = 0
# df.index = pd.to_datetime(v)
# df.index.name = 'date'
# df.drop(m.keys(), axis= 1, inplace=True)
df.head()

,mm,day,yyyy,HH_UTC,MM,Wind Speed-1 (2.83 m) (m/s),Wind Speed-2 (5.66 m) (m/s),Wind Direction (Deg),z0,ol,ustar,sigmav,Zm_1,Zm_2
0,1,2,2009,17,0,1.35,1.7,359,1.191100e-04,4.9684,0.042488,0.13,2.83,5.66
1,1,2,2009,17,2,0.35,1,346,1.156200e+02,1.9909,0.032712,0.01,2.83,5.66
2,1,2,2009,19,5,0.25,0.95,334,2.500000e+08,0.5227,0.009729,0.27,2.83,5.66
3,1,2,2009,20,0,0.60,1.5,358,1.137300e+01,4.3953,0.096159,0.64,2.83,5.66
4,1,5,2009,14,0,1.85,2.2,280,6.710600e-02,7.1336,0.107720,0.08,2.83,5.66


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3459 entries, 0 to 3458
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   mm                            3459 non-null   int64  
 1   day                           3459 non-null   int64  
 2   yyyy                          3459 non-null   int64  
 3   HH_UTC                        3459 non-null   int64  
 4   MM                            3459 non-null   int64  
 5    Wind Speed-1 (2.83 m) (m/s)  3459 non-null   float64
 6    Wind Speed-2 (5.66 m) (m/s)  3459 non-null   object 
 7    Wind Direction (Deg)         3459 non-null   int64  
 8   z0                            3459 non-null   float64
 9   ol                            3459 non-null   float64
 10  ustar                         3459 non-null   float64
 11  sigmav                        3459 non-null   float64
 12  Zm_1                          3459 non-null   float64
 13  Zm_

In [9]:
# not used here but could be used to create georeferenced rasters as in AmeriFlux example
lat,long = 36.438847, -121.373706

# Calculate footprint images

Assumes boundary layer height of 2 km. The footprints in this example are only calculated for hours between 7 AM and 8 PM, this alongwith footprint array domain, resolution, and other options are all easily adjustable using the arguments to the Kljun et al. footprint function.

In [12]:
# domain parameters
origin_d = 75.
dx = 3.
# only doing 2 days half hourly
for i, index in enumerate(df.loc['7/2/2009':'7/4/2009'].index):
    tmp = df.loc[index,:].astype(float)
    r = ffp.ffp_climatology(
        zm=tmp.zm - tmp.d,
        z0=tmp.z0,
        umean=tmp.u_mean,
        h=2000,
        ol=tmp.L,
        sigmav=tmp.sigma_v,
        ustar=tmp.u_star,
        wind_dir=tmp.wind_dir,
        fig=True,
        domain=[-origin_d,origin_d,-origin_d,origin_d],
        dx=dx,
        dy=dx,
        crop=True,
        verbosity=1
    )
    fig = r.get('fig')
    # fig.savefig('./celery_out/{}.png'.format(str(tmp.name).replace(':','-').replace(' ','_')))
        

# View an example footprint image

In [13]:
from IPython.display import Image
Image(filename='celery'
               '_out/2019-07-02_11-00-00.png') 

FileNotFoundError: [Errno 2] No such file or directory: 'celery_out/2019-07-02_11-00-00.png'